<a href="https://colab.research.google.com/github/cadairhinojosa/SCM-Practice-Cassandra-Adair-Hinojosa/blob/main/DangImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate --quiet
!pip install ultralytics --quiet
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00


In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from ultralytics import YOLO
import gradio as gr
from PIL import Image
import torch


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load BLIP-2 for rich, detailed image captions
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model_caption = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    device_map="auto",
    torch_dtype=torch.float16
)

# Load YOLOv8 for object detection
model_yolo = YOLO("yolov8n.pt")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

100%|██████████| 6.25M/6.25M [00:00<00:00, 101MB/s]


In [ ]:
def generate_caption(image):
    prompt = "Describe this image with full detail. Mention people, clothing, actions, setting, objects, and possible crime scene indicators."

    inputs = processor(images=image, text=prompt, return_tensors="pt")
    inputs = {k: v.to(model_caption.device, dtype=torch.float16) if v.dtype == torch.float32 else v.to(model_caption.device) for k, v in inputs.items()}

    output = model_caption.generate(**inputs, max_new_tokens=100)
    return processor.decode(output[0], skip_special_tokens=True)


In [ ]:
crime_words = ['police', 'gun', 'tape', 'chalk', 'blood', 'weapon', 'forensics', 'crime scene', 'evidence', 'shooting']

def detect_objects(image):
    results = model_yolo(image)
    labels = []
    for r in results:
        labels += [model_yolo.names[int(c)] for c in r.boxes.cls]
    return labels

def analyze_image(img):
    caption = generate_caption(img).lower()
    objects = detect_objects(img)

    matched = [word for word in crime_words if word in caption]

    if matched:
        status = f"🔴 Possible Crime Scene: {', '.join(set(matched))}"
    elif any(obj.lower() in ['gun', 'knife', 'blood'] for obj in objects):
        status = f"🟡 Caution: {', '.join(set(objects))}"
    else:
        status = "🟢 Safe"

    return f"Caption: {caption}\n\nSafety Status: {status}"


In [ ]:
gr.Interface(
    fn=analyze_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🧠 Crime Scene Caption & Detection App",
    description="Upload an image to receive a detailed caption and danger level."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26673b18913fc3bbab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
